In [2]:
# MIS 581
# Capstone - Whether Quality of Institution or Field of Study Better Predicts Student Loan Repayment
# Graham Burns
# Quality of Institution Analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# Load the dataset
data_path = '/content/Most-Recent-Cohorts-Institution.csv'
data = pd.read_csv(data_path, low_memory=False)

# Select relevant columns from dataset
columns = ['RPY_7YR_RT', 'COSTT4_A', 'C150_4']
data = data[columns].dropna()

# Rename columns for clarity
data.rename(columns={'RPY_7YR_RT': 'RPY_7YR_RT', 'COSTT4_A': 'cost_of_attendance',
                     'C150_4': 'graduation_rate'}, inplace=True)

# Cleaning the Data
# Convert categorical variable to numeric and inspect unique values in cost_of_attendance
print("First 10 unique values before cleaning:", data['cost_of_attendance'].unique()[:10])

# Remove rows where cost_of_attendance is non-numeric
data = data[pd.to_numeric(data['cost_of_attendance'], errors='coerce').notnull()]
print("Unique values in cost_of_attendance before cleaning:", data['cost_of_attendance'].unique())

# Remove invalid non-numeric values
data = data[~data['cost_of_attendance'].astype(str).str.contains('[^0-9.]', regex=True)]

# Ensure cost_of_attendance is numeric
data['cost_of_attendance'] = pd.to_numeric(data['cost_of_attendance'], errors='coerce')

# Remove non-numeric and invalid values
data = data[pd.to_numeric(data['cost_of_attendance'], errors='coerce').notnull()]

# Drop rows with NaN values after conversion
data = data.dropna(subset=['cost_of_attendance'])


# Scale numeric features
scaler = MinMaxScaler()
numeric_features = ['graduation_rate']

# Identify and remove or replace non-numeric values in numeric_features
for feature in numeric_features:
    #Replace 'PrivacySuppressed' and 'PS' with NaN
    data[feature] = data[feature].replace(['PrivacySuppressed', 'PS'], np.nan)
    # Convert the column to numeric
    data[feature] = pd.to_numeric(data[feature], errors='coerce')

# Drop rows with NaN values after cleaning
data = data.dropna(subset=numeric_features)

data[numeric_features] = scaler.fit_transform(data[numeric_features])



# Define the target variable - Loan Repayment
# Convert 'RPY_7YR_RT' to numeric before comparison, handling errors
data['RPY_7YR_RT'] = pd.to_numeric(data['RPY_7YR_RT'], errors='coerce')

# Drop rows with NaN values in 'RPY_7YR_RT' after conversion
data = data.dropna(subset=['RPY_7YR_RT'])
data['loan_repaid'] = (data['RPY_7YR_RT'] > 0.5).astype(int)

# Split dataset
X = data.drop(columns=['RPY_7YR_RT', 'loan_repaid'])
y = data['loan_repaid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Train Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"{model_name} ROC AUC: {roc_auc_score(y_true, y_pred):.3f}\n")

evaluate_model(y_test, y_pred_logreg, "Logistic Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")

# Feature importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Top Features:")
print(feature_importances.head(10))

# Define target variable
# Convert 'RPY_7YR_RT' to numeric before comparison, handling errors
data['RPY_7YR_RT'] = pd.to_numeric(data['RPY_7YR_RT'], errors='coerce')
# Drop rows with NaN values in 'RPY_7YR_RT' after conversion
data = data.dropna(subset=['RPY_7YR_RT'])
data['loan_repaid'] = (data['RPY_7YR_RT'] > 0.5).astype(int)  # Binary target

# Split dataset
X = data.drop(columns=['RPY_7YR_RT', 'loan_repaid'])
y = data['loan_repaid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Train Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"{model_name} ROC AUC: {roc_auc_score(y_true, y_pred):.3f}\n")

evaluate_model(y_test, y_pred_logreg, "Logistic Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")

# Feature importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Top Features:")
print(feature_importances.head(10))

First 10 unique values before cleaning: [23167. 26257. 25777. 21900. 31024. 19771. 33650. 35495. 32195. 36169.]
Unique values in cost_of_attendance before cleaning: [23167. 26257. 25777. ... 42461. 21407. 10184.]
Logistic Regression Accuracy: 0.824
Logistic Regression Precision: 0.841
Logistic Regression Recall: 0.975
Logistic Regression ROC AUC: 0.501

Random Forest Accuracy: 0.841
Random Forest Precision: 0.872
Random Forest Recall: 0.950
Random Forest ROC AUC: 0.607

Top Features:
              Feature  Importance
1     graduation_rate    0.509352
0  cost_of_attendance    0.490648
Logistic Regression Accuracy: 0.824
Logistic Regression Precision: 0.841
Logistic Regression Recall: 0.975
Logistic Regression ROC AUC: 0.501

Random Forest Accuracy: 0.841
Random Forest Precision: 0.872
Random Forest Recall: 0.950
Random Forest ROC AUC: 0.607

Top Features:
              Feature  Importance
1     graduation_rate    0.509352
0  cost_of_attendance    0.490648


In [3]:
# MIS 581
# Capstone - Whether Quality of Institution or Field of Study Better Predicts Student Loan Repayment
# Graham Burns
# Quality of Institution Analysis against other Factors

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# Load dataset
data_path = '/content/Most-Recent-Cohorts-Institution.csv'
data = pd.read_csv(data_path, low_memory=False)

# Select relevant columns
columns = ['RPY_7YR_RT', 'COSTT4_A', 'MD_EARN_WNE_P10', 'GRAD_DEBT_MDN', 'FAMINC', 'PAR_ED_PCT_1STGEN', 'C150_4']
data = data[columns].dropna()

# Rename them for clarity
data.rename(columns={'RPY_7YR_RT': 'RPY_7YR_RT', 'COSTT4_A': 'cost_of_attendance', 'FAMINC': 'family_income', 'PAR_ED_PCT_1STGEN': 'percent_first_gen',
                     'MD_EARN_WNE_P10': 'median_earnings', 'GRAD_DEBT_MDN': 'graduate_median_debt', 'C150_4': 'graduation_rate'}, inplace=True)

# Convert categorical variables to numeric and Inspect unique values in cost_of_attendance
print("First 10 unique values before cleaning:", data['cost_of_attendance'].unique()[:10])

# Remove rows where cost_of_attendance is non-numeric
data = data[pd.to_numeric(data['cost_of_attendance'], errors='coerce').notnull()]
print("Unique values in cost_of_attendance before cleaning:", data['cost_of_attendance'].unique())

# Remove invalid non-numeric values
data = data[~data['cost_of_attendance'].astype(str).str.contains('[^0-9.]', regex=True)]

# Ensure cost_of_attendance is numeric
data['cost_of_attendance'] = pd.to_numeric(data['cost_of_attendance'], errors='coerce')

# Remove non-numeric and invalid values
data = data[pd.to_numeric(data['cost_of_attendance'], errors='coerce').notnull()]

# Drop rows with NaN values after conversion
data = data.dropna(subset=['cost_of_attendance'])


# Scale numeric features
scaler = MinMaxScaler()
numeric_features = ['median_earnings', 'graduate_median_debt', 'family_income', 'percent_first_gen', 'graduation_rate']

# Identify and remove or replace non-numeric values in numeric_features
for feature in numeric_features:
    # Replace 'PrivacySuppressed' and 'PS' with NaN
    data[feature] = data[feature].replace(['PrivacySuppressed', 'PS'], np.nan)
    # Convert the column to numeric
    data[feature] = pd.to_numeric(data[feature], errors='coerce')

# Drop rows with NaN values after cleaning
data = data.dropna(subset=numeric_features)

data[numeric_features] = scaler.fit_transform(data[numeric_features])


# Define the target variable - Loan Repayment
# Convert 'RPY_7YR_RT' to numeric before comparison, handling errors
data['RPY_7YR_RT'] = pd.to_numeric(data['RPY_7YR_RT'], errors='coerce')

# Drop rows with NaN values in 'RPY_7YR_RT' after conversion
data = data.dropna(subset=['RPY_7YR_RT'])
data['loan_repaid'] = (data['RPY_7YR_RT'] > 0.5).astype(int)

# Split dataset
X = data.drop(columns=['RPY_7YR_RT', 'loan_repaid'])
y = data['loan_repaid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression Model
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)

# Train Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"{model_name} ROC AUC: {roc_auc_score(y_true, y_pred):.3f}\n")

evaluate_model(y_test, y_pred_logreg, "Logistic Regression")
evaluate_model(y_test, y_pred_rf, "Random Forest")

# Feature importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Top Features:")
print(feature_importances.head(10))

First 10 unique values before cleaning: [23167. 26257. 25777. 21900. 31024. 19771. 33650. 35495. 32195. 36169.]
Unique values in cost_of_attendance before cleaning: [23167. 26257. 25777. ... 25463. 22061. 15893.]


<ipython-input-3-2922d894afa2>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loan_repaid'] = (data['RPY_7YR_RT'] > 0.5).astype(int)  # Binary target


Logistic Regression Accuracy: 0.920
Logistic Regression Precision: 0.940
Logistic Regression Recall: 0.971
Logistic Regression ROC AUC: 0.764

Random Forest Accuracy: 0.926
Random Forest Precision: 0.973
Random Forest Recall: 0.941
Random Forest ROC AUC: 0.878

Top Features:
                Feature  Importance
3         family_income    0.336159
1       median_earnings    0.307299
5       graduation_rate    0.102128
4     percent_first_gen    0.101431
2  graduate_median_debt    0.084883
0    cost_of_attendance    0.068098
